In [33]:
from IPython.display import display, Math

# Define the boolean expression
clauses = [(-1,-3,-2), (1,2,-3), (3,2,1)]
#clauses = [(1,3,2), (-1,2), (-2, 3)]
# clauses = [(1,2), (2,3), (1, 3)]
#clauses = [(1, 2, 3), (1, 2, -3), (1, -2, 3), (1, -2, -3), (-1, 2, 3), (-1, 2, -3), (-1, -2, 3)]

# Get a list of variables
variables = set()
for clause in clauses:
    for variable in clause:
        variables.add(abs(variable))

print("List of variables:")
display(Math(', '.join(f'x{i}' for i in sorted(variables))))

# Print the boolean expression
latex_clauses = []
for clause in clauses:
    clause_literals = [f'\\overline{{x{abs(lit)}}}' if lit < 0 else f'x{abs(lit)}' for lit in clause]
    latex_clause = '(' + ' \\lor '.join(clause_literals) + ')'
    latex_clauses.append(latex_clause)
    
print("Given boolean expression:", end='')
display(Math(' \\land '.join(latex_clauses)))

List of variables:


<IPython.core.display.Math object>

Given boolean expression:

<IPython.core.display.Math object>

In [34]:
# Converting a 3SAT expression to k-coloring:

# First (variables) amount of nodes will be the true-literals,
# second (variables) amoun will be the false-literals,
# each 3 after that will represent a clause
num_literals = len(variables)
literals = list(variables)
edges = []
for index, variable in enumerate(literals):
    edge = (index, index+num_literals)
    edges.append(edge)
literals_offset = num_literals*2
for index, clause in enumerate(clauses):
    clause_offset = index*3
    total_offset = literals_offset + clause_offset
    edge1 = (total_offset, total_offset+1)
    edge2 = (total_offset+1, total_offset+2)
    edge3 = (total_offset, total_offset+2)
    edges.append(edge1)
    edges.append(edge2)
    edges.append(edge3)
    for index, literal in enumerate(clause):
        inbound_edge = total_offset+index
        literalIndex = abs(literal)
        if (literal < 0):
            literal_node_offset = num_literals
        else:
            literal_node_offset = 0
        outbound_edge = literal_node_offset + literalIndex
        edges.append((inbound_edge, outbound_edge))
print(edges)

[(0, 3), (1, 4), (2, 5), (6, 7), (7, 8), (6, 8), (6, 4), (7, 6), (8, 5), (9, 10), (10, 11), (9, 11), (9, 1), (10, 2), (11, 6), (12, 13), (13, 14), (12, 14), (12, 3), (13, 2), (14, 1)]


In [36]:
import itertools
# Converting the 3SAT expression to a cubic polynomial
# Calculate the constant term, linear terms and quadratic terms in the composite penalty function

constant_term = 0
linear_terms = {}
quadratic_terms = {}
cubic_terms = {}

for clause in clauses:
    variables = []
    # Order them in such that any non-negated literals appear first
    for literal in clause:
        if (literal > 0):
            variables.append(abs(literal))
    for literal in clause:
        if (literal < 0):
            variables.append(abs(literal))
    vartuple = tuple(sorted(variables))
    num_negatives = len([literal for literal in clause if literal < 0])
    if num_negatives == 0:
        constant_term += 1
        for var in variables:
            linear_terms[var] = linear_terms.get(var, 0) - 1
        for pair in itertools.combinations(variables, 2):
            quadratic_terms[tuple(sorted(list(pair)))] = quadratic_terms.get(tuple(sorted(list(pair))), 0) + 1
        cubic_terms[vartuple] = cubic_terms.get(vartuple, 0) - 1
    elif num_negatives == 1:
        linear_terms[variables[2]] = linear_terms.get(variables[2], 0) + 1
        quadratic_terms[tuple(sorted(list((variables[0], variables[2]))))] = quadratic_terms.get(tuple(sorted(list((variables[0], variables[2])))), 0) - 1
        quadratic_terms[tuple(sorted(list((variables[1], variables[2]))))] = quadratic_terms.get(tuple(sorted(list((variables[1], variables[2])))), 0) - 1
        cubic_terms[vartuple] = cubic_terms.get(vartuple, 0) + 1
    elif num_negatives == 2:
        quadratic_terms[tuple(sorted(list((variables[1], variables[2]))))] = quadratic_terms.get(tuple(sorted(list((variables[1], variables[2])))), 0) + 1
        cubic_terms[vartuple] = cubic_terms.get(vartuple, 0) - 1
    else:
        cubic_terms[vartuple] = cubic_terms.get(vartuple, 0) + 1

print("Constant term: ", constant_term)
print("Linear terms: ", sorted(linear_terms.items()))
print("Quadratic terms: ", sorted(quadratic_terms.items()))
print("Cubic terms: ", sorted(cubic_terms.items()))

Constant term:  1
Linear terms:  [(1, -1), (2, -1), (3, 0)]
Quadratic terms:  [((1, 2), 1), ((1, 3), 0), ((2, 3), 0)]
Cubic terms:  [((1, 2, 3), 1)]
